In [8]:
import os
import random
import math
from Crypto.Random import get_random_bytes
from Crypto.Cipher import AES, PKCS1_OAEP
from Crypto.Protocol.KDF import PBKDF2
from Crypto.Util.Padding import pad, unpad
from Crypto.PublicKey import RSA

def generate_rsa_keys():
    key = RSA.generate(2048)
    private_key = key.export_key()
    public_key = key.publickey().export_key()
    return private_key, public_key

def RSA_encrypt(message, public_key):
    rsa_key = RSA.import_key(public_key)
    cipher_rsa = PKCS1_OAEP.new(rsa_key)
    encrypted_message = cipher_rsa.encrypt(message)
    return encrypted_message

def RSA_decrypt(cipher_text, private_key):
    rsa_key = RSA.import_key(private_key)
    cipher_rsa = PKCS1_OAEP.new(rsa_key)
    decrypted_message = cipher_rsa.decrypt(cipher_text)
    return decrypted_message

def encrypt_file(file_path, public_key, chunk_size, encrypted_filepath):
    # Generate the AES key
    key = get_random_bytes(32)

    # Splitting into chunks and encrypting
    with open(file_path, 'rb') as f:
        cnt = 0
        while True:
            chunk = f.read(chunk_size)
            if not chunk:
                break

            # AES Encryption
            cipher = AES.new(key, AES.MODE_CBC)
            ciphered_message = cipher.encrypt(pad(chunk, AES.block_size))

            # RSA encryption of the AES key
            encrypted_AES_key = RSA_encrypt(key, public_key)

            # Saving the encrypted data
            with open(f'{file_path}.chunk{cnt}.aeskey', 'wb') as key_file:
                key_file.write(encrypted_AES_key)

            with open(f'{file_path}.chunk{cnt}.aesmessage', 'wb') as data_file:
                data_file.write(cipher.iv)
                data_file.write(ciphered_message)

            with open(encrypted_filepath , 'wb') as encrypted_file : 
                encrypted_file.write(ciphered_message)
            cnt += 1

    return cnt

def decrypt_file(file_path, private_key, chunk_count, output_file_path):
    decrypted_chunks = []
    for i in range(chunk_count):
        with open(f'{file_path}.chunk{i}.aeskey', 'rb') as key_file:
            encrypted_aes_key = key_file.read()
        aes_key = RSA_decrypt(encrypted_aes_key, private_key)

        with open(f'{file_path}.chunk{i}.aesmessage', 'rb') as data_file:
            iv = data_file.read(16)
            cipher_data = data_file.read()
        cipher = AES.new(aes_key, AES.MODE_CBC, iv=iv)
        decrypted_data = unpad(cipher.decrypt(cipher_data), AES.block_size)

        decrypted_chunks.append(decrypted_data)

    with open(output_file_path, 'wb') as output_file:
        for chunk in decrypted_chunks:
            output_file.write(chunk)

    for i in range(chunk_count):
        os.remove(f'{file_path}.chunk{i}.aeskey')
        os.remove(f'{file_path}.chunk{i}.aesmessage')

def main():
    # RSA keys
    private_key, public_key = generate_rsa_keys()

    # Dealing with files
    file_path = 'plaintext.txt'
    encrypted_filepath = 'ciphertext.txt'
    decrypted_filepath = 'decrypted.txt'

    # Chunk size = 1Kb
    chunk_size = 1024

    # Splitting the large file and encrypting
    chunk_count = encrypt_file(file_path, public_key, chunk_size,encrypted_filepath)
    print(f'File {file_path} encrypted into {chunk_count} chunks successfully!')

    # Decrypting the file
    decrypt_file(file_path, private_key, chunk_count, decrypted_filepath)
    print(f'File decrypted successfully into {decrypted_filepath}.')

if __name__ == "__main__":
    main()


File plaintext.txt encrypted into 2 chunks successfully!
File decrypted successfully into decrypted.txt.
